In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from trainlib.FileCollection import FileCollection
from trainlib.config import Config
import trainlib.cuts as cuts
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import copy
import re
from scipy import interpolate
import scipy.integrate as integrate

Welcome to JupyROOT 6.10/09


In [3]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
#candidate_branches = ["PFMET", "nCleanedJetsPt30", "nCleanedJetsPt30BTagged_bTagSF", "nExtraLep", "ZZMass", "nExtraZ", "Z1Mass", "Z2Mass", "Z1Pt", "Z2Pt", "ZZMassErr", "ZZPt", "ZZEta", "ZZPhi", "Z1Flav", "Z2Flav", "costhetastar", "helphi", "helcosthetaZ1", "helcosthetaZ2", "phistarZ1", "phistarZ2", "xi", "xistar"]
candidate_branches = ["PFMET", "nCleanedJetsPt30", "nCleanedJetsPt30BTagged_bTagSF", "nExtraLep", "ZZMass", "nExtraZ", "Z1Mass", "Z2Mass", "Z1Pt", "Z2Pt", "ZZMassErr", "ZZPt", "ZZEta", "ZZPhi", "Z1Flav", "Z2Flav"]
MELA_branches = ["D_VBF2j_ggH_ME", "D_VBF1j_ggH_ME", "D_WHh_ggH_ME", "D_ZHh_ggH_ME", "D_WHh_ZHh_ME", "D_VBF2j_WHh_ME", "D_VBF2j_ZHh_ME"]
#list_branches = ["Jet", "Lep", "ExtraLep"]
list_branches = ["Jet", "ExtraLep"]
pt_limits = [30.0, 0.0, 0.0]

In [5]:
allbranches = ["JetPt", "JetEta", "JetPhi", "LepPt", "LepEta", "LepPhi", "ExtraLepPt", "ExtraLepEta", "ExtraLepPhi"] + candidate_branches + MELA_branches + ["LHEAssociatedParticleId", "GenAssocLep1Id", "GenAssocLep2Id", "training_weight"]

In [6]:
#MC_path = "/data_CMS/cms/wind/CJLST_NTuples_randomizeda/"
MC_path = "/data_CMS/cms/wind/CJLST_NTuples/"

In [7]:
def WHhadr0j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] == 0

def WHhadr01j_cut(row):
    return cuts.WHhadr_cut(row) and (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def WHhadr1j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] == 1

def WHhadr2j_cut(row):
    return cuts.WHhadr_cut(row) and row["nCleanedJetsPt30"] >= 2

def ZHhadr0j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] == 0

def ZHhadr01j_cut(row):
    return cuts.ZHhadr_cut(row) and (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def ZHhadr1j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] == 1

def ZHhadr2j_cut(row):
    return cuts.ZHhadr_cut(row) and row["nCleanedJetsPt30"] >= 2

def mZZ0j_cut(row):
    return cuts.mZZ_cut(row) and row["nCleanedJetsPt30"] == 0

def mZZ01j_cut(row):
    return cuts.mZZ_cut(row) and (row["nCleanedJetsPt30"] == 0 or row["nCleanedJetsPt30"] == 1)

def mZZ1j_cut(row):
    return cuts.mZZ_cut(row) and row["nCleanedJetsPt30"] == 1

def mZZ2j_cut(row):
    return cuts.mZZ_cut(row) and row["nCleanedJetsPt30"] >= 2

In [8]:
collections = {"VBF2j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ2j_cut},
            "VBF1j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ1j_cut},
            "VBF0j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ0j_cut},
            "VBF01j": {MC_path + "VBFH125/ZZ4lAnalysis.root": mZZ01j_cut},
            "VBF": {MC_path + "VBFH125/ZZ4lAnalysis.root": cuts.mZZ_cut},
            "ggH2j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ2j_cut},
            "ggH1j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ1j_cut},
            "ggH0j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ0j_cut},
            "ggH01j": {MC_path + "ggH125/ZZ4lAnalysis.root": mZZ01j_cut},
            "ggH" : {MC_path + "ggH125/ZZ4lAnalysis.root": cuts.mZZ_cut},
            "WHh2j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr2j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr2j_cut},
            "WHh1j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr1j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr1j_cut},
            "WHh0j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr0j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr0j_cut},
            "WHh": {MC_path + "WplusH125/ZZ4lAnalysis.root": cuts.WHhadr_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": cuts.WHhadr_cut},
            "WHh01j": {MC_path + "WplusH125/ZZ4lAnalysis.root": WHhadr01j_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": WHhadr01j_cut},
            "WHl": {MC_path + "WplusH125/ZZ4lAnalysis.root": cuts.WHlept_cut, MC_path + "WminusH125/ZZ4lAnalysis.root": cuts.WHlept_cut},
            "ZHh2j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr2j_cut},
            "ZHh1j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr1j_cut},
            "ZHh01j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr01j_cut},
            "ZHh0j": {MC_path + "ZH125/ZZ4lAnalysis.root": ZHhadr0j_cut},
            "ZHh": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHhadr_cut},
            "ZHl": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHlept_cut},
            "ttHh": {MC_path + "ttH125/ZZ4lAnalysis.root": cuts.ttHhadr_cut},
            "ttHl": {MC_path + "ttH125/ZZ4lAnalysis.root": cuts.ttHlept_cut},
            "ZHMET": {MC_path + "ZH125/ZZ4lAnalysis.root": cuts.ZHMET_cut}
          }

In [9]:
# all the model combinations for which neural networks are currently trained
discriminant_pairs = [("VBF2j", "ggH2j"), ("VBF1j", "ggH1j"), ("VBF0j", "ggH0j"), ("WHh2j", "ggH2j"), 
                     ("WHh1j", "ggH1j"), ("WHh0j", "ggH0j"), ("ZHh2j", "ggH2j"), ("ZHh1j", "ggH1j"), 
                      ("ZHh0j", "ggH0j"), ("WHh2j", "ZHh2j"), ("WHh01j", "ZHh01j"), ("VBF2j", "WHh2j"),
                     ("VBF1j", "WHh1j"), ("VBF0j", "WHh0j"), ("VBF2j", "ZHh2j"), ("VBF1j", "ZHh1j"), 
                      ("VBF0j", "ZHh0h"), ("WHl", "ggH"), ("WHl", "VBF"), ("WHl", "WHh"), ("WHl", "ZHh"),
                     ("WHl", "ZHl"), ("WHl", "ZHMET"), ("WHl", "ttHh"), ("WHl", "ttHl"), ("ZHh", "ZHl"),
                     ("ZHh", "ZHMET"), ("ZHh", "ttHh"), ("ZHh", "ttHl"), ("ZHl", "ggH"), ("ZHl", "VBF"),
                     ("ZHl", "WHh"), ("ZHl", "ZHMET"), ("ZHl", "ttHh"), ("ZHl", "ttHl"), ("ZHMET", "ggH"),
                     ("ZHMET", "VBF"), ("ZHMET", "WHh"), ("ZHMET", "ttHh"), ("ZHMET", "ttHl"), ("ttHh", "ggH"),
                      ("ttHh", "VBF"), ("ttHh", "WHh"), ("ttHh", "ttHl"), ("ttHl", "ggH"), ("ttHl", "VBF"),
                     ("ttHl", "WHh")]

In [10]:
def extract_order(df, col_basename, sorted_column, columns, order):
    def get_index(row, order, col_basename, sorted_column):
        sorted_column = row[col_basename + sorted_column]
        if order >= len(sorted_column):
            return -1
        else:
            return np.flipud(np.argsort(sorted_column))[order]
    
    index_column = pd.DataFrame(df.transform(lambda row: get_index(row, order, col_basename, sorted_column), axis = 1, raw = True))
    index_column.columns = ["index"]
    df_temp = pd.concat([index_column, df], axis = 1)
    
    def get_element(row, column_name):
        if row["index"] == -1:
            return 0
        else:
            return row[column_name][row["index"]]
        
    extracted_cols = pd.DataFrame()
    for column in columns:
        extracted_col = pd.DataFrame(df_temp.transform(lambda row: get_element(row, col_basename + column), axis = 1, raw = True))
        extracted_col.columns = [col_basename + column + "_" + str(order)]
        extracted_cols = pd.concat([extracted_cols, extracted_col], axis = 1)
        
    return extracted_cols

In [11]:
def prepare_data(df, col_basenames, sorted_column, columns, orders, pt_limits):
    all_extracted = pd.DataFrame()
    for col_basename, pt_limit in zip(col_basenames, pt_limits):
        for order in orders:
            extracted = extract_order(df, col_basename, sorted_column, columns, order)
            mask = extracted[col_basename + "Pt_" + str(order)] < pt_limit
            extracted[mask] = 0.0

            all_extracted = pd.concat([all_extracted, extracted], axis = 1)
            
    return all_extracted

In [12]:
def get_data(H1_coll, H0_coll, read_branches, input_branches, list_branches, pt_limits):
    H1_df = H1_coll.get_data(read_branches, 0.0, 1.0)
    H0_df = H0_coll.get_data(read_branches, 0.0, 1.0)
    
    H1_list_df = prepare_data(H1_df, list_branches, "Pt", ["Pt", "Eta", "Phi"], range(2), pt_limits)
    H0_list_df = prepare_data(H0_df, list_branches, "Pt", ["Pt", "Eta", "Phi"], range(2), pt_limits)
    
    list_branches_unrolled = H1_list_df.columns
            
    H1_df = pd.concat([H1_df, H1_list_df], axis = 1)
    H0_df = pd.concat([H0_df, H0_list_df], axis = 1)        
    
    complete_input_branches = np.concatenate([input_branches, list_branches_unrolled])
            
    H1_df = H1_df[complete_input_branches]
    H0_df = H0_df[complete_input_branches]
    
    return H1_df, H0_df

In [13]:
def get_data_dmatrix(H1_coll, H0_coll, read_branches, input_branches, list_branches, pt_limits):
    H1_df, H0_df = get_data(H1_coll, H0_coll, read_branches, input_branches, list_branches, pt_limits)
    
    complete_input_branches = H1_df.columns
    print "number of input variables: " + str(len(complete_input_branches))
    print "final list of inputs: " + str(complete_input_branches)
    
    H1_data = H1_df.as_matrix()
    H0_data = H0_df.as_matrix()
    H1_target = np.ones(np.shape(H1_data)[0])
    H0_target = np.zeros(np.shape(H0_data)[0])
    
    target = np.concatenate([H1_target, H0_target])
    data = np.concatenate([H1_data, H0_data])
    
    dmatrix = xgb.DMatrix(data, label = target, feature_names = complete_input_branches)
    
    return dmatrix

In [14]:
def get_feature_correlation(source, corr_branches, mandatory_branches, optional_branches, list_branches, pt_limits):    
    coll = FileCollection(collections[source], 0.0, 0.5)
    
    input_branches = [branch for branch in mandatory_branches]
    
    for optional_branch in optional_branches:
        if "0j" in source and ("0j" in optional_branch):
            input_branches.append(optional_branch)
            
        if "1j" in source and ("1j" in optional_branch):
            input_branches.append(optional_branch)
            
        if "2j" in source and ("2j" in optional_branch):
            input_branches.append(optional_branch)

    df, _ = get_data(coll, coll, allbranches, input_branches, list_branches, pt_limits)

    df = df[corr_branches]
    
    fig = plt.figure(figsize=(10,15))
    ax = fig.add_subplot(111)
    
    cax = ax.matshow(df.corr(), vmin = -1.0, vmax = 1.0, cmap = "RdBu")
    
    fig.colorbar(cax)
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    ax.set_yticklabels([''] + corr_branches)
    ax.set_xticklabels([''] + corr_branches, rotation = 'vertical')
    
    return fig

In [15]:
def get_interpolating_function(data, bins):
    bin_centers = [np.mean([bins[i], bins[i + 1]]) for i in range(len(bins) - 1)]
    intf = interpolate.interp1d(bin_centers, data, kind = "linear")
    interpolated_function = lambda x: intf(x) if x > bin_centers[0] and x < bin_centers[-1] else 0
    
    return interpolated_function

In [16]:
def get_binned_data(df, branch):
    data = df[branch].as_matrix()
    weights = df["training_weight"].as_matrix()
    
    # set the bin width
    q75, q25 = np.percentile(data, [75, 25])
    bin_width = max(2 * (q75 - q25) / len(data)**0.33, 0.005)

    data_max = np.max(data)
    data_min = np.min(data)
    bins = np.arange(data_min, data_max + bin_width, bin_width)
    
    weights = weights / (np.sum(weights) * bin_width)
    
    hist = np.histogram(data, bins = bins, weights = weights)
    return hist

In [17]:
def get_feature_importance_list_separation(disc_pair, mandatory_branches, optional_branches, list_branches, pt_limits):
    H1_name = disc_pair[0]
    H0_name = disc_pair[1]
    
    input_branches = [branch for branch in mandatory_branches]
    
    for optional_branch in optional_branches:
        if ("0j" in H1_name or "0j" in H0_name) and ("0j" in optional_branch):
            input_branches.append(optional_branch)  
        elif ("1j" in H1_name or "1j" in H0_name) and ("1j" in optional_branch):
            input_branches.append(optional_branch)
        elif ("2j" in H1_name or "2j" in H0_name) and not ("1j" in optional_branch):
            input_branches.append(optional_branch)
            
    # needed to build the histograms
    input_branches.append("training_weight")
    
    H1_coll = FileCollection(collections[H1_name], 0.0, 0.5)
    H0_coll = FileCollection(collections[H0_name], 0.0, 0.5)
    
    H1_df, H0_df = get_data(H1_coll, H0_coll, allbranches, input_branches, list_branches, pt_limits)
    
    available_branches = H1_df.columns
    
    implist = {}
    for branch in available_branches:
                
        if "training_weight" not in branch:
            data_H1, bins_H1 = get_binned_data(H1_df, branch)
            data_H0, bins_H0 = get_binned_data(H0_df, branch)

            if(len(data_H1) > 0):
                H1_func = get_interpolating_function(data_H1, bins_H1)
                H0_func = get_interpolating_function(data_H0, bins_H0)

                # compute the separation in this branch
                global_min = np.min(np.concatenate([bins_H0, bins_H1]))
                global_max = np.max(np.concatenate([bins_H0, bins_H1]))

                separation_func = lambda x: (H1_func(x) + H0_func(x)) * (H1_func(x) - H0_func(x))**2

                sep = integrate.quad(separation_func, global_min, global_max)[0]
            else:
                sep = 0.0
                
            print "separation for " + branch + " = " + str(sep)
            implist[branch] = sep
        
    # normalize the importance list
    impsum = sum([val for key, val in implist.iteritems()])
    for key in implist.keys():
        implist[key] /= impsum
        
    return None, None, implist

In [18]:
def get_feature_importance_list_BDT(disc_pair, mandatory_branches, optional_branches, list_branches, pt_limits):
    H1_name = disc_pair[0]
    H0_name = disc_pair[1]
    
    # first assemble the list of branches that can serve as input: it will *always* contain the mandatory branches,
    # and *can* contain some of the optional branches, if the name of the categories allows it
    input_branches = [branch for branch in mandatory_branches]
    
    for optional_branch in optional_branches:
        if ("0j" in H1_name or "0j" in H0_name) and ("0j" in optional_branch):
            input_branches.append(optional_branch)  
        elif ("1j" in H1_name or "1j" in H0_name) and ("1j" in optional_branch):
            input_branches.append(optional_branch)
        elif ("2j" in H1_name or "2j" in H0_name) and not ("1j" in optional_branch):
            input_branches.append(optional_branch)
            
        # the fully inclusive categories (i.e. those with NO "xxj" in their name, can not use MELA, since there may
        # be events with low number of jets contained)
    
    # get the training data for the BDT ...
    H1_coll_train = FileCollection(collections[H1_name], 0.0, 0.5)
    H0_coll_train = FileCollection(collections[H0_name], 0.0, 0.5)
    
    dtrain = get_data_dmatrix(H1_coll_train, H0_coll_train, allbranches, input_branches, list_branches, pt_limits)
    
    # ... and the validation data as well
    H1_coll_val = FileCollection(collections[H1_name], 0.5, 1.0)
    H0_coll_val = FileCollection(collections[H0_name], 0.5, 1.0)
    dval = get_data_dmatrix(H1_coll_val, H0_coll_val, allbranches, input_branches, list_branches, pt_limits)
    
    evallist = [(dtrain, 'train'), (dval, 'eval')]
    
    # perform the training
    # try different tree depths and choose the one that gives the best RMSE (i.e. avoid too deep trees to start with)
        
    params = {'eta': 0.01, 'silent': 1, 'gamma': 0.5, 'objective': 'binary:logistic'}
    params['nthread'] = 4
    params['eval_metric'] = 'rmse'
    max_num_rounds = 2000
    
    best_loss = 1e6
    best_imp = None
    best_params = None
    for tree_depth in range(1,8):
        params['max_depth'] = tree_depth
        
        bst = xgb.train(params, dtrain, max_num_rounds, evals = evallist, early_stopping_rounds = 10, verbose_eval = False)
    
        pred = bst.predict(dval)
        cur_loss = np.sqrt(mean_squared_error(pred, dval.get_label()))
        cur_imp = bst.get_fscore()

        print "for max_depth = " + str(params['max_depth']) + ": loss = " + str(cur_loss)
        
        if cur_loss < best_loss:
            best_loss = cur_loss
            best_imp = copy.copy(cur_imp)
            best_params = copy.copy(params)
            
    # normalize the usage score w.r.t. the total score (i.e. sum of all individuals)
    score_sum = sum([val for key, val in best_imp.iteritems()])
    used_variables = {key: val / float(score_sum) for key, val in sorted(best_imp.iteritems(), key = lambda x: x[1], reverse = True)}           
    return best_params, dtrain.feature_names, used_variables

In [19]:
def get_histogram(df, branch, label):
    data = df[branch].as_matrix()
    weights = df["training_weight"].as_matrix()
    
    # set the bin width
    q75, q25 = np.percentile(data, [75, 25])
    bin_width = max(2 * (q75 - q25) / len(data)**0.33, 0.005)

    data_max = np.max(data)
    data_min = np.min(data)
    bins = np.arange(data_min, data_max + bin_width, bin_width)
    
    weights = weights / (np.sum(weights) * bin_width)
    
    fig = plt.hist(data, bins = bins, weights = weights, alpha = 0.5, label = label)
    return fig

In [20]:
def plot_branch(disc_pair, branch, start_fraction = 0.0, end_fraction = 1.0):
    H1_name = disc_pair[0]
    H0_name = disc_pair[1]
    
    # get the training data for the BDT ...
    H1_coll = FileCollection(collections[H1_name], start_fraction, end_fraction)
    H0_coll = FileCollection(collections[H0_name], start_fraction, end_fraction)
    
    H1_df, H0_df = get_data(H1_coll, H0_coll, allbranches, allbranches, list_branches, pt_limits)
    
    plt.figure()
    H1_hist = get_histogram(H1_df, branch, H1_name)
    H0_hist = get_histogram(H0_df, branch, H0_name)
    
    plt.legend(loc = 'upper right')
    plt.show()

In [21]:
def plot_variables(discs):
    plotframe = pd.DataFrame()
    
    for disc in discs:
        _, _, implist = get_feature_importance_list_BDT(disc, candidate_branches, MELA_branches, list_branches, pt_limits)
        
        # cut the list to select only the 95% most important variables
        cutimplist = {key: val for key, val in implist.iteritems() if val > 0.00}
        curframe = pd.DataFrame(cutimplist, index = [len(plotframe)])
        
        plotframe = pd.concat([plotframe, curframe])
        
    plotframe = plotframe.fillna(0.0)
    
    print plotframe
    print "number of pre-selected input variables = " + str(len(plotframe.columns))
    
    # start the plotting
    parameters = plotframe.columns
    plotdata = np.transpose(plotframe.as_matrix())
    
    plt.close('all')
    fig = plt.figure(figsize=(10,15))
    ax = fig.add_subplot(111)
    cax = ax.matshow(plotdata, cmap = 'Blues')
    
    # make axis labels
    disclabels = []
    for disc in discs:
        if "0j" in disc[0] or "0j" in disc[1]:
            disclabels.append('D_' + re.sub('0j', '', disc[0]) + "_" + re.sub('0j', '', disc[1]) + "_0j")
        elif "01j" in disc[0] or "01j" in disc[1]:
            disclabels.append('D_' + re.sub('01j', '', disc[0]) + "_" + re.sub('01j', '', disc[1]) + "_01j")
        elif "1j" in disc[0] or "1j" in disc[1]:
            disclabels.append('D_' + re.sub('1j', '', disc[0]) + "_" + re.sub('1j', '', disc[1]) + "_1j")
        elif "2j" in disc[0] or "2j" in disc[1]:
            disclabels.append('D_' + re.sub('2j', '', disc[0]) + "_" + re.sub('2j', '', disc[1]) + "_2j")
        else:
            disclabels.append('D_' + disc[0] + "_" + disc[1] + "_2j")
            
    disclabels = np.concatenate([[''], np.array(disclabels)])
    parameters = np.concatenate([[''], np.array(parameters)])
        
    ax.set_xticklabels(disclabels, rotation = 'vertical')
    ax.set_yticklabels(parameters)
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    # sort the used variables according to their importance
    sorted_implist = []
    for key, val in sorted(cutimplist.iteritems(), key = lambda x: x[1], reverse = True):
        sorted_implist.append((key, val))
    
    return fig, sorted_implist

In [22]:
%%capture
fig, implist = plot_variables([("VBF2j", "ggH2j")])

In [23]:
implist

[('D_VBF2j_ggH_ME', 0.0934156378600823),
 ('JetPt_0', 0.08994708994708994),
 ('JetPt_1', 0.07360376249265138),
 ('ZZPt', 0.059376837154614934),
 ('nCleanedJetsPt30', 0.04838330393885949),
 ('PFMET', 0.046325690770135215),
 ('D_WHh_ZHh_ME', 0.046325690770135215),
 ('Z2Pt', 0.0435626102292769),
 ('D_ZHh_ggH_ME', 0.042328042328042326),
 ('Z1Pt', 0.042034097589653145),
 ('JetEta_1', 0.03968253968253968),
 ('JetPhi_1', 0.03827160493827161),
 ('ZZEta', 0.036331569664902995),
 ('JetEta_0', 0.035155790711346266),
 ('D_VBF2j_WHh_ME', 0.03403880070546737),
 ('ZZMass', 0.03368606701940035),
 ('ZZPhi', 0.031099353321575544),
 ('JetPhi_0', 0.030393885949441504),
 ('Z1Mass', 0.029218106995884775),
 ('D_VBF2j_ZHh_ME', 0.027865961199294534),
 ('Z2Mass', 0.025279247501469725),
 ('ZZMassErr', 0.024926513815402705),
 ('D_WHh_ggH_ME', 0.024573780129335684),
 ('Z1Flav', 0.001763668430335097),
 ('Z2Flav', 0.0014697236919459142),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0009406231628453851)]

In [24]:
%%capture
fig, implist = plot_variables([("VBF1j", "ggH1j")])

In [25]:
implist

[('JetEta_0', 0.21530531324345759),
 ('ZZPt', 0.09448283675087799),
 ('D_VBF1j_ggH_ME', 0.09227370567576754),
 ('JetPt_0', 0.08468335787923417),
 ('ZZEta', 0.07584683357879234),
 ('PFMET', 0.07465730146142517),
 ('ZZMassErr', 0.05392545598731166),
 ('Z1Pt', 0.052509346323779314),
 ('Z2Pt', 0.043049733771383256),
 ('ZZMass', 0.0417469128809335),
 ('JetPhi_0', 0.04135040217514444),
 ('Z2Mass', 0.04101053585589668),
 ('Z1Mass', 0.04038744760394245),
 ('ZZPhi', 0.03460972017673049),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.00708054831766172),
 ('ExtraLepPt_0', 0.0030021524866885693),
 ('ExtraLepPhi_0', 0.0019825535289452814),
 ('Z2Flav', 0.0012461765039084626),
 ('Z1Flav', 0.0005664438654129376),
 ('ExtraLepEta_0', 0.00016993315962388127),
 ('nExtraLep', 0.00011328877308258752)]

In [26]:
%%capture
fig, implist = plot_variables([("VBF0j", "ggH0j")])

In [27]:
implist

[('ZZPt', 0.6882453151618398),
 ('PFMET', 0.07722884724588303),
 ('Z1Pt', 0.06700738216922203),
 ('ZZPhi', 0.05281090289608177),
 ('ZZEta', 0.04713231118682567),
 ('Z2Pt', 0.020442930153321975),
 ('Z2Mass', 0.016467915956842702),
 ('ZZMass', 0.009653605905735378),
 ('nExtraLep', 0.0073821692220329355),
 ('ZZMassErr', 0.005110732538330494),
 ('ExtraLepEta_0', 0.004542873367404884),
 ('Z1Mass', 0.0034071550255536627),
 ('ExtraLepPt_0', 0.0005678591709256105)]

In [28]:
%%capture
fig, implist = plot_variables([("WHh2j", "ggH2j")])

In [29]:
implist

[('JetPt_0', 0.0807279063014229),
 ('JetPt_1', 0.07763043267834673),
 ('D_WHh_ggH_ME', 0.07075791307714645),
 ('ZZPt', 0.06732165327654632),
 ('D_VBF2j_ggH_ME', 0.06451456780563353),
 ('JetEta_0', 0.05793243635659665),
 ('ZZEta', 0.0492207917916949),
 ('Z1Pt', 0.04176749588616784),
 ('PFMET', 0.04138031168328332),
 ('ZZPhi', 0.03949278869422128),
 ('JetPhi_0', 0.03881521633917336),
 ('ZZMass', 0.03818604200948601),
 ('ZZMassErr', 0.037653663730519794),
 ('D_WHh_ZHh_ME', 0.03465298615816475),
 ('nCleanedJetsPt30', 0.03450779208208305),
 ('JetEta_1', 0.03247507501693931),
 ('Z2Mass', 0.031507114509728006),
 ('Z2Pt', 0.029232407317781435),
 ('JetPhi_1', 0.02836124286129126),
 ('Z1Mass', 0.028312844835930693),
 ('D_ZHh_ggH_ME', 0.02424741070564321),
 ('D_VBF2j_ZHh_ME', 0.023860226502758687),
 ('D_VBF2j_WHh_ME', 0.020617558803600812),
 ('Z2Flav', 0.003484657825960701),
 ('Z1Flav', 0.0019843190397831768),
 ('ExtraLepPt_0', 0.0010163585325718711),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0003387

In [30]:
%%capture
fig, implist = plot_variables([("WHh1j", "ggH1j")])

In [31]:
implist

[('JetEta_0', 0.13644053459651498),
 ('ZZEta', 0.11850786668922349),
 ('JetPt_0', 0.09896802571476908),
 ('D_VBF1j_ggH_ME', 0.08340382338013873),
 ('ZZPt', 0.07793379574804038),
 ('PFMET', 0.06087520442113573),
 ('ZZMassErr', 0.058647718942085376),
 ('ZZPhi', 0.05292392714148762),
 ('Z2Mass', 0.0519652625049343),
 ('Z2Pt', 0.0519652625049343),
 ('ZZMass', 0.05182428241132352),
 ('JetPhi_0', 0.05145773416793549),
 ('Z1Pt', 0.04649523487283595),
 ('Z1Mass', 0.043985789206564034),
 ('Z2Flav', 0.004680539107877968),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.004116618733434839),
 ('ExtraLepPt_0', 0.0029041899283821122),
 ('Z1Flav', 0.002875993909659956),
 ('nExtraLep', 2.819601872215643e-05)]

In [32]:
%%capture
fig, implist = plot_variables([("WHh0j", "ggH0j")])

In [33]:
implist

[('ZZPt', 0.48711484593837534),
 ('ZZEta', 0.13165266106442577),
 ('ZZMassErr', 0.08263305322128851),
 ('Z2Pt', 0.05546218487394958),
 ('Z2Mass', 0.05266106442577031),
 ('Z1Pt', 0.050140056022408966),
 ('PFMET', 0.046218487394957986),
 ('ZZMass', 0.031932773109243695),
 ('Z1Mass', 0.025210084033613446),
 ('ZZPhi', 0.02492997198879552),
 ('ExtraLepPt_0', 0.011204481792717087),
 ('ExtraLepEta_0', 0.0008403361344537816)]

In [34]:
%%capture
fig, implist = plot_variables([("ZHh2j", "ggH2j")])

In [35]:
implist

[('JetPt_0', 0.10333591931730023),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.09107835531419706),
 ('JetPt_1', 0.07757951900698215),
 ('D_VBF2j_ggH_ME', 0.07183863460046548),
 ('ZZPt', 0.07168347556245151),
 ('ZZEta', 0.07013188518231186),
 ('D_ZHh_ggH_ME', 0.06966640806826997),
 ('D_WHh_ggH_ME', 0.0660977501939488),
 ('JetEta_0', 0.05120248254460822),
 ('nCleanedJetsPt30', 0.04127230411171451),
 ('Z1Pt', 0.038789759503491075),
 ('D_VBF2j_WHh_ME', 0.03413498836307215),
 ('JetPhi_0', 0.02916989914662529),
 ('PFMET', 0.02591155934833204),
 ('D_WHh_ZHh_ME', 0.021722265321955005),
 ('JetEta_1', 0.020325833979829323),
 ('ZZPhi', 0.01970519782777347),
 ('ZZMass', 0.01768813033359193),
 ('Z1Mass', 0.01768813033359193),
 ('ZZMassErr', 0.015671062839410395),
 ('D_VBF2j_ZHh_ME', 0.01334367726920093),
 ('Z2Pt', 0.012102404965089216),
 ('JetPhi_1', 0.009464701318851822),
 ('Z2Mass', 0.008688906128782002),
 ('Z2Flav', 0.0017067494181536075)]

In [36]:
%%capture
fig, implist = plot_variables([("ZHh1j", "ggH1j")])

In [37]:
implist

[('ZZEta', 0.12485887665819927),
 ('JetEta_0', 0.12461191081004798),
 ('D_VBF1j_ggH_ME', 0.10277307366638443),
 ('JetPt_0', 0.09624611910810048),
 ('ZZPt', 0.07285492520462884),
 ('ZZPhi', 0.06593988145639289),
 ('PFMET', 0.05895427603725656),
 ('ZZMass', 0.0558848433530906),
 ('Z2Mass', 0.05486169912503528),
 ('ZZMassErr', 0.04784081287044877),
 ('JetPhi_0', 0.045441716059836296),
 ('Z2Pt', 0.04360711261642676),
 ('Z1Pt', 0.04304261924922382),
 ('Z1Mass', 0.04067880327406153),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.012912785774767146),
 ('Z2Flav', 0.004939316963025684),
 ('ExtraLepPt_0', 0.0026460626587637595),
 ('Z1Flav', 0.001905165114309907)]

In [38]:
%%capture
fig, implist = plot_variables([("ZHh0j", "ggH0j")])

In [39]:
implist

[('ZZPt', 0.3042638538107265),
 ('Z1Pt', 0.1209329965829743),
 ('ZZEta', 0.10117367404546129),
 ('Z2Mass', 0.0901797652651909),
 ('Z1Mass', 0.08423711187045016),
 ('ZZPhi', 0.08082008616847422),
 ('PFMET', 0.05571237557569455),
 ('Z2Pt', 0.048432625167137124),
 ('ZZMassErr', 0.03223889466646858),
 ('ZZMass', 0.03179319566186302),
 ('ExtraLepPt_0', 0.029861833308572276),
 ('ExtraLepEta_0', 0.014262368147377804),
 ('ExtraLepPhi_0', 0.004011291041450008),
 ('Z2Flav', 0.0016342296835537068),
 ('Z1Flav', 0.00044569900460555636)]

In [40]:
%%capture
fig, implist = plot_variables([("WHh2j", "ZHh2j")])

In [41]:
implist

[('D_WHh_ZHh_ME', 0.12430568856540893),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.11166443210113006),
 ('D_ZHh_ggH_ME', 0.06473855583221605),
 ('JetPhi_1', 0.06205707718827811),
 ('PFMET', 0.05190576517908447),
 ('nCleanedJetsPt30', 0.051522696801379046),
 ('JetEta_0', 0.044435931813828766),
 ('D_VBF2j_WHh_ME', 0.04386132924727064),
 ('ZZMass', 0.04117985060333269),
 ('JetPt_1', 0.04079678222562728),
 ('Z2Mass', 0.03485922237119326),
 ('ZZEta', 0.033518483049224286),
 ('ZZPt', 0.032560812104960736),
 ('Z1Pt', 0.030262401838728213),
 ('JetEta_1', 0.029879333461022794),
 ('JetPt_0', 0.029687799272170082),
 ('ZZMassErr', 0.02643171806167401),
 ('D_WHh_ggH_ME', 0.02585711549511588),
 ('D_VBF2j_ZHh_ME', 0.025282512928557748),
 ('Z2Pt', 0.02489944455085233),
 ('JetPhi_0', 0.023367171040030645),
 ('Z1Mass', 0.01647194024133308),
 ('D_VBF2j_ggH_ME', 0.010151312009193642),
 ('Z2Flav', 0.00995977782034093),
 ('ZZPhi', 0.00881057268722467),
 ('Z1Flav', 0.0015322735108216816)]

In [42]:
%%capture
fig, implist = plot_variables([("WHh01j", "ZHh01j")])

In [43]:
implist

[('nCleanedJetsPt30BTagged_bTagSF', 0.20610687022900764),
 ('ZZMass', 0.12213740458015267),
 ('ZZPt', 0.10347752332485156),
 ('ZZEta', 0.09245122985581),
 ('JetPt_0', 0.07633587786259542),
 ('D_VBF1j_ggH_ME', 0.06446140797285835),
 ('ZZPhi', 0.043256997455470736),
 ('PFMET', 0.04240882103477523),
 ('Z2Mass', 0.039016115351993216),
 ('JetEta_0', 0.0364715860899067),
 ('Z1Mass', 0.03307888040712468),
 ('Z2Pt', 0.031382527565733676),
 ('JetPhi_0', 0.029686174724342665),
 ('Z1Pt', 0.020356234096692113),
 ('ExtraLepEta_0', 0.019508057675996608),
 ('ZZMassErr', 0.016963528413910092),
 ('ExtraLepPt_0', 0.01272264631043257),
 ('ExtraLepPhi_0', 0.006785411365564037),
 ('Z1Flav', 0.0033927056827820186)]

In [44]:
%%capture
fig, implist = plot_variables([("VBF2j", "WHh2j")])

In [45]:
implist

[('D_VBF2j_ggH_ME', 0.12462303366207515),
 ('nCleanedJetsPt30', 0.10864781155758416),
 ('D_WHh_ggH_ME', 0.07906104817018501),
 ('JetEta_0', 0.05917352677479827),
 ('ZZEta', 0.05265302795663868),
 ('JetPt_0', 0.0506968783111908),
 ('JetEta_1', 0.049474284782785885),
 ('D_WHh_ZHh_ME', 0.048496209960061945),
 ('D_VBF2j_ZHh_ME', 0.044665416904393185),
 ('ZZPt', 0.04238324231803733),
 ('JetPt_1', 0.03455864373624582),
 ('PFMET', 0.029016219740810173),
 ('ZZPhi', 0.028934713505583177),
 ('Z1Mass', 0.028038144918086233),
 ('D_VBF2j_WHh_ME', 0.02730458880104328),
 ('ZZMass', 0.0269785638601353),
 ('D_ZHh_ggH_ME', 0.025429945390822398),
 ('ZZMassErr', 0.02461488303855245),
 ('JetPhi_0', 0.024370364332871465),
 ('Z1Pt', 0.02428885809764447),
 ('Z2Mass', 0.022006683511288615),
 ('JetPhi_1', 0.018012877985165866),
 ('Z2Pt', 0.01776835927948488),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.004727361643165702),
 ('Z2Flav', 0.0017116309397668921),
 ('ExtraLepPt_0', 0.0016301247045398973),
 ('Z1Flav', 0.000

In [46]:
%%capture
fig, implist = plot_variables([("VBF1j", "WHh1j")])

In [47]:
implist

[('JetEta_0', 0.1589121338912134),
 ('ZZEta', 0.13893305439330544),
 ('JetPt_0', 0.1002092050209205),
 ('D_VBF1j_ggH_ME', 0.06527196652719665),
 ('PFMET', 0.058744769874476986),
 ('ZZMassErr', 0.05784518828451883),
 ('Z2Mass', 0.05713389121338912),
 ('ZZPt', 0.055230125523012555),
 ('JetPhi_0', 0.05489539748953975),
 ('Z1Pt', 0.05179916317991632),
 ('Z2Pt', 0.051401673640167365),
 ('ZZPhi', 0.05069037656903766),
 ('ZZMass', 0.047740585774058576),
 ('Z1Mass', 0.04117154811715481),
 ('Z2Flav', 0.004435146443514644),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0027615062761506275),
 ('Z1Flav', 0.0021548117154811717),
 ('ExtraLepPt_0', 0.0006694560669456067)]

In [48]:
%%capture
fig, implist = plot_variables([("VBF0j", "WHh0j")])

In [49]:
implist

[('ZZEta', 0.23030303030303031),
 ('PFMET', 0.21363636363636362),
 ('ZZMassErr', 0.2015151515151515),
 ('Z2Pt', 0.11818181818181818),
 ('Z1Pt', 0.09242424242424242),
 ('ExtraLepPt_0', 0.04242424242424243),
 ('Z1Mass', 0.03484848484848485),
 ('ZZPhi', 0.02727272727272727),
 ('Z2Mass', 0.022727272727272728),
 ('ZZMass', 0.012121212121212121),
 ('ZZPt', 0.004545454545454545)]

In [50]:
%%capture
fig, implist = plot_variables([("VBF2j", "ZHh2j")])

In [51]:
implist

[('D_VBF2j_ggH_ME', 0.09419977592002068),
 ('nCleanedJetsPt30', 0.06683616306127725),
 ('ZZEta', 0.06295785572696716),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.05416702576919762),
 ('JetEta_0', 0.050116349220029305),
 ('PFMET', 0.04572093424114453),
 ('JetEta_1', 0.045419288115142634),
 ('D_ZHh_ggH_ME', 0.043566319055416705),
 ('D_VBF2j_WHh_ME', 0.04261828837369646),
 ('JetPt_1', 0.04210118072912178),
 ('JetPt_0', 0.039989657847108506),
 ('D_WHh_ggH_ME', 0.03964491941739205),
 ('JetPhi_1', 0.036973196587089543),
 ('D_WHh_ZHh_ME', 0.034818581401361716),
 ('Z1Pt', 0.03395673532707059),
 ('ZZMassErr', 0.03356890459363958),
 ('D_VBF2j_ZHh_ME', 0.03300870464535034),
 ('ZZMass', 0.032534689304490216),
 ('Z2Mass', 0.031931397052486425),
 ('ZZPhi', 0.030121520296475048),
 ('JetPhi_0', 0.028397828147892785),
 ('ZZPt', 0.028009997414461776),
 ('Z1Mass', 0.023743859346720676),
 ('Z2Pt', 0.022494182538998535),
 ('Z1Flav', 0.0019391536671550462),
 ('Z2Flav', 0.0011634922002930277)]

In [52]:
%%capture
fig, implist = plot_variables([("VBF1j", "ZHh1j")])

In [53]:
implist

[('JetEta_0', 0.16034299590226134),
 ('ZZEta', 0.15538523802296758),
 ('JetPt_0', 0.08979612485455557),
 ('PFMET', 0.07115394344108868),
 ('D_VBF1j_ggH_ME', 0.06189608944199929),
 ('ZZPt', 0.05688774219659028),
 ('ZZMassErr', 0.05602772297263115),
 ('Z2Pt', 0.05567359740982446),
 ('ZZPhi', 0.051778216218950776),
 ('ZZMass', 0.05104467041027976),
 ('Z2Mass', 0.04811048717559569),
 ('Z1Pt', 0.04026913542773309),
 ('JetPhi_0', 0.038802043810391054),
 ('Z1Mass', 0.03852380229675722),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.014468558708959377),
 ('Z2Flav', 0.005868366469368139),
 ('nExtraLep', 0.0022512267921282947),
 ('Z1Flav', 0.0015429756665148986),
 ('ExtraLepPt_0', 0.00015176809834572774),
 ('ExtraLepEta_0', 2.5294683057621287e-05)]

In [54]:
%%capture
fig, implist = plot_variables([("VBF0j", "ZHh0j")])

In [55]:
implist

[('ExtraLepPt_0', 0.20869565217391303),
 ('PFMET', 0.15072463768115943),
 ('Z1Pt', 0.13043478260869565),
 ('Z2Pt', 0.12173913043478261),
 ('ZZEta', 0.11884057971014493),
 ('Z1Mass', 0.08985507246376812),
 ('ZZPt', 0.0782608695652174),
 ('ZZMassErr', 0.06956521739130435),
 ('ZZPhi', 0.03188405797101449)]

In [56]:
%%capture
fig, implist = plot_variables([("WHl", "ggH")])

In [57]:
implist

[('PFMET', 0.13346332089974003),
 ('ZZPt', 0.12111450209110433),
 ('JetPt_0', 0.10246411212840512),
 ('ZZEta', 0.0761840171809653),
 ('ZZPhi', 0.05948344071436645),
 ('Z2Mass', 0.0553012320560642),
 ('Z1Pt', 0.053634000226065334),
 ('ZZMass', 0.0506386345653894),
 ('ZZMassErr', 0.05027127839945744),
 ('JetEta_0', 0.048745337402509324),
 ('JetPhi_0', 0.04747371990505256),
 ('Z1Mass', 0.04422403074488527),
 ('Z2Pt', 0.04074827625183678),
 ('ExtraLepPt_0', 0.038346332089974004),
 ('JetPt_1', 0.01593760596812479),
 ('ExtraLepEta_0', 0.015570249802192834),
 ('JetEta_1', 0.014835537470328924),
 ('JetPhi_1', 0.011416299310500734),
 ('nCleanedJetsPt30', 0.0072623488188086355),
 ('ExtraLepPhi_0', 0.0054255679891488644),
 ('Z2Flav', 0.0031083983271165366),
 ('Z1Flav', 0.0030801401605063863),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0012433593308466147),
 ('nExtraLep', 2.8258166610150335e-05)]

In [58]:
%%capture
fig, implist = plot_variables([("WHl", "VBF")])

In [59]:
implist

[('PFMET', 0.14425190194420964),
 ('JetEta_0', 0.1275993237531699),
 ('ZZEta', 0.09712595097210482),
 ('JetPt_0', 0.08520710059171598),
 ('JetEta_1', 0.060228233305156385),
 ('ExtraLepEta_0', 0.05270498732037194),
 ('ExtraLepPt_0', 0.047083685545224005),
 ('Z1Pt', 0.04590025359256129),
 ('ZZMassErr', 0.04420963651732882),
 ('ZZPt', 0.04353338968723584),
 ('ZZPhi', 0.0382502113271344),
 ('ZZMass', 0.03495350803043111),
 ('JetPhi_0', 0.03410819949281488),
 ('Z2Mass', 0.03144547759932375),
 ('Z2Pt', 0.029754860524091292),
 ('Z1Mass', 0.02565511411665258),
 ('JetPt_1', 0.02510566356720203),
 ('JetPhi_1', 0.014285714285714285),
 ('nCleanedJetsPt30', 0.0060016906170752326),
 ('ExtraLepPhi_0', 0.0053254437869822485),
 ('Z2Flav', 0.0026204564666103126),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0022400676246830094),
 ('Z1Flav', 0.002197802197802198),
 ('nExtraLep', 0.00021132713440405747)]

In [60]:
%%capture
fig, implist = plot_variables([("WHl", "WHh")])

In [61]:
implist

[('PFMET', 0.22336275918702525),
 ('JetPt_0', 0.1038801067542599),
 ('ExtraLepPt_0', 0.09464175733935537),
 ('JetEta_0', 0.08163963594060084),
 ('ZZPt', 0.07062204886060357),
 ('JetPt_1', 0.04407034831998905),
 ('ZZEta', 0.04242797509067269),
 ('Z2Mass', 0.03880106754259906),
 ('Z1Pt', 0.037637719838499964),
 ('JetEta_1', 0.0318894135358927),
 ('ExtraLepEta_0', 0.030247040306576337),
 ('ZZPhi', 0.029083692602477246),
 ('ZZMassErr', 0.027372887155272703),
 ('Z1Mass', 0.023609115171422706),
 ('JetPhi_0', 0.023198521864093616),
 ('ZZMass', 0.023061657428317252),
 ('Z2Pt', 0.0225141996852118),
 ('JetPhi_1', 0.017381783343598165),
 ('nCleanedJetsPt30', 0.014233901320741805),
 ('ExtraLepPhi_0', 0.014097036884965441),
 ('nExtraLep', 0.003421610894409088),
 ('Z1Flav', 0.0017792376650927256),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0009580510504345446),
 ('Z2Flav', 6.843221788818175e-05)]

In [62]:
%%capture
fig, implist = plot_variables([("WHl", "ZHh")])

In [63]:
implist

[('PFMET', 0.2084490929890505),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.07092662199705835),
 ('JetPt_0', 0.06594214740970747),
 ('ExtraLepPt_0', 0.06398104265402843),
 ('JetEta_0', 0.06381761725772185),
 ('ZZPt', 0.052541264912567415),
 ('JetPhi_0', 0.04943618238274228),
 ('Z1Pt', 0.04649452524922373),
 ('ZZEta', 0.038568393528354304),
 ('ZZPhi', 0.03766955384866808),
 ('JetPt_1', 0.034973034809609414),
 ('Z2Pt', 0.03301193005393038),
 ('ZZMass', 0.03235822846870404),
 ('Z1Mass', 0.03113253799640464),
 ('nCleanedJetsPt30', 0.030397123713025005),
 ('ZZMassErr', 0.026556626899820232),
 ('nExtraLep', 0.022470991992155583),
 ('JetEta_1', 0.02132701421800948),
 ('ExtraLepEta_0', 0.020673312632783134),
 ('ExtraLepPhi_0', 0.01863049517895081),
 ('Z2Mass', 0.016015688838045432),
 ('JetPhi_1', 0.011194639647001145),
 ('Z2Flav', 0.0022062428501389117),
 ('Z1Flav', 0.0012256904722993953)]

In [64]:
%%capture
fig, implist = plot_variables([("WHl", "ZHl")])

In [65]:
implist

[('ZZMass', 0.10978588877712316),
 ('PFMET', 0.08933275853810413),
 ('Z2Pt', 0.08190832016094267),
 ('Z2Mass', 0.0745796809886478),
 ('Z1Mass', 0.06811323466015232),
 ('JetPt_0', 0.06111989270489055),
 ('ZZEta', 0.0489054940843991),
 ('ZZMassErr', 0.046941610384633806),
 ('ZZPhi', 0.04502562628730181),
 ('ZZPt', 0.04148105570723763),
 ('ExtraLepPt_0', 0.040427264453705036),
 ('JetEta_0', 0.03985246922450544),
 ('ExtraLepEta_0', 0.03448771375197586),
 ('Z1Pt', 0.03180533601571107),
 ('JetPhi_1', 0.02706327537481439),
 ('JetPhi_0', 0.025386789289648895),
 ('ExtraLepPhi_0', 0.020884226660918714),
 ('JetPt_1', 0.018824543756286824),
 ('JetEta_1', 0.018297648129520526),
 ('nExtraLep', 0.015040475164056138),
 ('nExtraZ', 0.01494467595918954),
 ('ExtraLepPt_1', 0.010537912535325957),
 ('nCleanedJetsPt30', 0.009723619293959861),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.008430330028260765),
 ('ExtraLepEta_1', 0.006322747521195574),
 ('ExtraLepPhi_1', 0.005987450304162476),
 ('Z2Flav', 0.0032092733

In [66]:
%%capture
fig, implist = plot_variables([("WHl", "ZHMET")])

In [67]:
implist

[('PFMET', 0.166690782810013),
 ('Z1Pt', 0.08594993488641296),
 ('ZZPt', 0.08146433222399074),
 ('JetPt_0', 0.08045145420344378),
 ('ExtraLepPt_0', 0.0797279699030531),
 ('ZZMass', 0.07017797713789611),
 ('ZZEta', 0.0570105628707857),
 ('ZZPhi', 0.05585298799016061),
 ('Z2Mass', 0.0549848068296918),
 ('Z2Pt', 0.048762841846331936),
 ('ZZMassErr', 0.0470264795253943),
 ('Z1Mass', 0.039212849081174936),
 ('JetEta_0', 0.035161336998987124),
 ('JetPhi_0', 0.025756041093908263),
 ('nExtraLep', 0.018955288670235857),
 ('JetEta_1', 0.015916654608594993),
 ('JetPt_1', 0.01172044566632904),
 ('ExtraLepEta_0', 0.007668933584141224),
 ('JetPhi_1', 0.0059325712632035885),
 ('Z1Flav', 0.0036174215019534077),
 ('nCleanedJetsPt30', 0.003472724641875271),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.003472724641875271),
 ('Z2Flav', 0.0010128780205469542)]

In [68]:
%%capture
fig, implist = plot_variables([("WHl", "ttHh")])

In [69]:
implist

[('PFMET', 0.13342525752291345),
 ('ExtraLepPt_0', 0.09789926190283073),
 ('JetPt_0', 0.09222159136994079),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.08281288020115175),
 ('nCleanedJetsPt30', 0.07526968935031227),
 ('JetPt_1', 0.06918647092221591),
 ('JetEta_1', 0.05564117122232136),
 ('JetEta_0', 0.053370103009165386),
 ('ZZEta', 0.03868926920269284),
 ('Z2Mass', 0.036255981831454294),
 ('JetPhi_0', 0.035201557303917594),
 ('ZZMassErr', 0.031957174142266204),
 ('JetPhi_1', 0.028469462243490957),
 ('ZZPhi', 0.026441722767458837),
 ('ZZMass', 0.02636061318841755),
 ('ZZPt', 0.02514396950279828),
 ('Z1Pt', 0.024251764133344147),
 ('Z1Mass', 0.021980695920188174),
 ('Z2Pt', 0.019628518127990914),
 ('nExtraLep', 0.007462081271798199),
 ('ExtraLepPhi_0', 0.007218752534674345),
 ('ExtraLepEta_0', 0.005920999270013789),
 ('Z1Flav', 0.004623246005353232),
 ('Z2Flav', 0.0005677670532889934)]

In [70]:
%%capture
fig, implist = plot_variables([("WHl", "ttHl")])

In [71]:
implist

[('nCleanedJetsPt30BTagged_bTagSF', 0.11859994214636968),
 ('JetPt_0', 0.09444605148973098),
 ('nCleanedJetsPt30', 0.08793751807925948),
 ('JetPt_1', 0.08027191206248192),
 ('PFMET', 0.0669655770899624),
 ('ExtraLepPt_1', 0.053514608041654615),
 ('ZZEta', 0.05163436505640729),
 ('Z1Mass', 0.04758461093433613),
 ('Z2Pt', 0.03991900491755858),
 ('ZZMass', 0.038328030083887764),
 ('nExtraLep', 0.03601388487127567),
 ('ZZPt', 0.03442291003760486),
 ('ExtraLepEta_0', 0.03109632629447498),
 ('JetEta_1', 0.029939253688168934),
 ('ZZPhi', 0.02936071738501591),
 ('JetEta_0', 0.027769742551345098),
 ('JetPhi_0', 0.02704657217240382),
 ('ExtraLepPt_0', 0.022996818050332657),
 ('Z2Mass', 0.01793462539774371),
 ('JetPhi_1', 0.016632918715649408),
 ('Z1Pt', 0.01547584610934336),
 ('ZZMassErr', 0.010702921608330923),
 ('Z1Flav', 0.008388776395718832),
 ('ExtraLepEta_1', 0.0054960948799537175),
 ('ExtraLepPhi_0', 0.00506219265258895),
 ('nExtraZ', 0.0013017066820943013),
 ('ExtraLepPhi_1', 0.000867804

In [72]:
%%capture
fig, implist = plot_variables([("ZHh", "ZHl")])

In [73]:
implist

[('ZZMass', 0.1165547542070795),
 ('PFMET', 0.10221337975627953),
 ('Z1Mass', 0.07974798972063334),
 ('JetPt_0', 0.07833872171101716),
 ('Z2Mass', 0.06573820774268424),
 ('Z2Pt', 0.06457763408770621),
 ('ExtraLepPt_0', 0.061095913122772114),
 ('ZZPt', 0.061095913122772114),
 ('ZZMassErr', 0.06018403382243223),
 ('ZZPhi', 0.046340048080908566),
 ('Z1Pt', 0.0450136781895051),
 ('JetEta_0', 0.04161485534278372),
 ('JetPhi_0', 0.031086794329768713),
 ('ZZEta', 0.03025781314764155),
 ('ExtraLepEta_0', 0.019646854016413827),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.01840338224322308),
 ('JetEta_1', 0.01666252176075603),
 ('JetPt_1', 0.015667744342203433),
 ('ExtraLepPhi_0', 0.014092680096161818),
 ('JetPhi_1', 0.01284920832297107),
 ('nExtraLep', 0.007875321230208074),
 ('nCleanedJetsPt30', 0.006217358865953743),
 ('Z2Flav', 0.004227804028848545),
 ('Z1Flav', 0.0004973887092762995)]

In [74]:
%%capture
fig, implist = plot_variables([("ZHh", "ZHMET")])

In [75]:
implist

[('PFMET', 0.3483293295479362),
 ('ZZPt', 0.1174929023804324),
 ('JetPt_0', 0.1028608866564752),
 ('JetEta_0', 0.0722865254422363),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.05350513212491811),
 ('nCleanedJetsPt30', 0.04149377593360996),
 ('Z1Pt', 0.03887311640096091),
 ('JetPt_1', 0.03843633981218607),
 ('ZZEta', 0.03647084516269928),
 ('Z2Pt', 0.030574361214238916),
 ('Z2Mass', 0.022930770910679188),
 ('Z1Mass', 0.02074688796680498),
 ('ZZPhi', 0.016379122079056563),
 ('ZZMass', 0.015287180607119458),
 ('ZZMassErr', 0.014195239135182355),
 ('JetPhi_0', 0.0104826381305962),
 ('ExtraLepPt_0', 0.009172308364271676),
 ('JetEta_1', 0.0058964839484603624),
 ('JetPhi_1', 0.0032758244158113126),
 ('Z2Flav', 0.001310329766324525)]

In [76]:
%%capture
fig, implist = plot_variables([("ZHh", "ttHh")])

In [77]:
implist

[('nCleanedJetsPt30', 0.1357981220657277),
 ('JetPt_1', 0.12570422535211268),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.11643192488262911),
 ('JetPt_0', 0.10586854460093896),
 ('PFMET', 0.07018779342723004),
 ('JetEta_1', 0.059154929577464786),
 ('ZZPt', 0.05669014084507042),
 ('Z1Pt', 0.05293427230046948),
 ('ZZEta', 0.048826291079812206),
 ('Z2Pt', 0.028051643192488263),
 ('JetPhi_1', 0.025469483568075116),
 ('Z1Mass', 0.02488262910798122),
 ('ZZMass', 0.022887323943661973),
 ('ExtraLepPt_0', 0.02112676056338028),
 ('ZZPhi', 0.019014084507042252),
 ('Z2Mass', 0.017605633802816902),
 ('JetEta_0', 0.01737089201877934),
 ('ZZMassErr', 0.016666666666666666),
 ('nExtraLep', 0.012206572769953052),
 ('ExtraLepPhi_0', 0.009859154929577466),
 ('JetPhi_0', 0.009507042253521126),
 ('Z1Flav', 0.003051643192488263),
 ('ExtraLepEta_0', 0.0007042253521126761)]

In [78]:
%%capture
fig, implist = plot_variables([("ZHh", "ttHl")])

In [79]:
implist

[('PFMET', 0.1856730769230769),
 ('nCleanedJetsPt30', 0.11009615384615384),
 ('ExtraLepPt_0', 0.08634615384615385),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.08365384615384615),
 ('ZZMass', 0.06076923076923077),
 ('JetPt_0', 0.05307692307692308),
 ('ZZPt', 0.04596153846153846),
 ('Z1Pt', 0.04471153846153846),
 ('ZZPhi', 0.04403846153846154),
 ('JetPt_1', 0.03875),
 ('Z2Pt', 0.03875),
 ('Z1Mass', 0.037307692307692306),
 ('ZZMassErr', 0.030576923076923078),
 ('ZZEta', 0.0225),
 ('Z2Mass', 0.01875),
 ('JetEta_1', 0.017115384615384616),
 ('ExtraLepEta_0', 0.016923076923076923),
 ('JetEta_0', 0.016442307692307694),
 ('JetPhi_0', 0.016057692307692308),
 ('nExtraLep', 0.015),
 ('JetPhi_1', 0.010673076923076922),
 ('ExtraLepPhi_0', 0.0047115384615384615),
 ('Z1Flav', 0.001346153846153846),
 ('Z2Flav', 0.0007692307692307692)]

In [80]:
%%capture
fig, implist = plot_variables([("ZHl", "ggH")])

In [81]:
implist

[('ZZMass', 0.11226415094339623),
 ('PFMET', 0.0880188679245283),
 ('Z1Mass', 0.0785377358490566),
 ('ZZPt', 0.07082547169811321),
 ('JetPt_0', 0.06877358490566038),
 ('ZZMassErr', 0.06577830188679246),
 ('Z2Pt', 0.06424528301886792),
 ('ZZEta', 0.06268867924528301),
 ('Z1Pt', 0.06148584905660377),
 ('JetEta_0', 0.0567688679245283),
 ('Z2Mass', 0.055235849056603774),
 ('ZZPhi', 0.048278301886792455),
 ('ExtraLepPt_0', 0.03370283018867924),
 ('JetPhi_0', 0.027452830188679247),
 ('JetEta_1', 0.02311320754716981),
 ('JetPt_1', 0.018207547169811322),
 ('JetPhi_1', 0.016014150943396227),
 ('ExtraLepEta_0', 0.014316037735849057),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.01025943396226415),
 ('nCleanedJetsPt30', 0.008113207547169812),
 ('Z2Flav', 0.006108490566037736),
 ('nExtraLep', 0.005141509433962264),
 ('ExtraLepPhi_0', 0.0024528301886792454),
 ('Z1Flav', 0.002216981132075472)]

In [82]:
%%capture
fig, implist = plot_variables([("ZHl", "VBF")])

In [83]:
implist

[('JetEta_0', 0.10692389710782353),
 ('ZZMass', 0.08349993335998934),
 ('PFMET', 0.08123417299746769),
 ('ZZEta', 0.07993469278955084),
 ('JetPt_0', 0.07626949220311875),
 ('Z1Mass', 0.061008929761428764),
 ('JetEta_1', 0.06020924963347994),
 ('ZZMassErr', 0.0508796481407437),
 ('ZZPt', 0.05027988804478209),
 ('Z2Pt', 0.047847527655604426),
 ('Z2Mass', 0.040983606557377046),
 ('ExtraLepPt_0', 0.0392842862854858),
 ('Z1Pt', 0.038984406237505),
 ('ZZPhi', 0.036118885779021725),
 ('JetPhi_0', 0.034552845528455285),
 ('ExtraLepEta_0', 0.03355324536851926),
 ('JetPhi_1', 0.01835932293749167),
 ('nCleanedJetsPt30', 0.015926962548314008),
 ('JetPt_1', 0.013927762228441956),
 ('nExtraLep', 0.012828202052512328),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.009329601492736239),
 ('ExtraLepPhi_0', 0.004564840730374517),
 ('Z2Flav', 0.0023324003731840597),
 ('Z1Flav', 0.0011662001865920299)]

In [84]:
%%capture
fig, implist = plot_variables([("ZHl", "WHh")])

In [85]:
implist

[('ZZMass', 0.11046304858098016),
 ('PFMET', 0.09911800270289495),
 ('Z2Mass', 0.07678355501813784),
 ('ZZMassErr', 0.07141332953979657),
 ('Z1Mass', 0.06689664983284728),
 ('JetPt_0', 0.06543850914005264),
 ('Z2Pt', 0.0651895582900633),
 ('ZZPt', 0.06273561419731133),
 ('Z1Pt', 0.05708087346183939),
 ('ExtraLepPt_0', 0.048083078455082154),
 ('JetEta_0', 0.046553808948004836),
 ('ZZEta', 0.04587808521231951),
 ('JetPhi_0', 0.03250586812717832),
 ('ZZPhi', 0.03247030372003699),
 ('JetPhi_1', 0.02084074258482111),
 ('JetEta_1', 0.020271712070559784),
 ('nExtraLep', 0.01443914929938118),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.014368020485098514),
 ('ExtraLepEta_0', 0.012981008606586528),
 ('JetPt_1', 0.012554235720890533),
 ('nCleanedJetsPt30', 0.0124119780923252),
 ('Z2Flav', 0.005583611921189274),
 ('ExtraLepPhi_0', 0.0034853118998506294),
 ('Z1Flav', 0.0024539440927519736)]

In [86]:
%%capture
fig, implist = plot_variables([("ZHl", "ZHMET")])

In [87]:
implist

[('PFMET', 0.13287841191066999),
 ('ZZMass', 0.12531017369727046),
 ('Z2Pt', 0.09094292803970223),
 ('ExtraLepPt_0', 0.08933002481389578),
 ('JetPt_0', 0.08238213399503722),
 ('Z2Mass', 0.07121588089330025),
 ('Z1Mass', 0.06898263027295286),
 ('Z1Pt', 0.055707196029776675),
 ('ZZMassErr', 0.04640198511166253),
 ('ZZPt', 0.04640198511166253),
 ('JetEta_0', 0.037096774193548385),
 ('JetPt_1', 0.029404466501240695),
 ('ZZPhi', 0.028411910669975185),
 ('ZZEta', 0.027667493796526055),
 ('JetPhi_0', 0.015384615384615385),
 ('nExtraLep', 0.013895781637717122),
 ('JetPhi_1', 0.010173697270471464),
 ('ExtraLepEta_0', 0.00880893300248139),
 ('nCleanedJetsPt30', 0.00794044665012407),
 ('JetEta_1', 0.004466501240694789),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0028535980148883377),
 ('Z2Flav', 0.0026054590570719605),
 ('Z1Flav', 0.0017369727047146402)]

In [88]:
%%capture
fig, implist = plot_variables([("ZHl", "ttHh")])

In [89]:
implist

[('JetPt_0', 0.11650023775558725),
 ('ZZMass', 0.11103185924869234),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.09486447931526391),
 ('Z2Pt', 0.0903471231573942),
 ('nCleanedJetsPt30', 0.08368996671421779),
 ('ExtraLepPt_0', 0.07037565382786495),
 ('JetPt_1', 0.06348074179743224),
 ('Z2Mass', 0.05444602948169282),
 ('nExtraLep', 0.05111745126010461),
 ('JetEta_1', 0.050879695672848314),
 ('Z1Mass', 0.046362339514978604),
 ('JetEta_0', 0.0461245839277223),
 ('PFMET', 0.03209700427960057),
 ('ZZPhi', 0.017118402282453638),
 ('ZZMassErr', 0.016405135520684736),
 ('JetPhi_0', 0.012363290537327628),
 ('ZZPt', 0.012363290537327628),
 ('Z1Pt', 0.010461245839277223),
 ('ZZEta', 0.008321445553970519),
 ('JetPhi_1', 0.0033285782215882074),
 ('ExtraLepEta_0', 0.002615311459819306),
 ('ExtraLepPt_1', 0.002615311459819306),
 ('Z1Flav', 0.0023775558725630053),
 ('ExtraLepPhi_0', 0.0007132667617689016)]

In [90]:
%%capture
fig, implist = plot_variables([("ZHl", "ttHl")])

In [91]:
implist

[('PFMET', 0.14089843045282338),
 ('JetPt_0', 0.10580912863070539),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0699981959227855),
 ('Z2Pt', 0.06566841060797401),
 ('ExtraLepPt_0', 0.06251127548259065),
 ('Z1Mass', 0.06233086776114018),
 ('ZZMass', 0.056828432256900596),
 ('nCleanedJetsPt30', 0.04916110409525528),
 ('Z2Mass', 0.041583979794335195),
 ('ZZPt', 0.0374346022009742),
 ('nExtraZ', 0.03382644777196464),
 ('ZZEta', 0.03184196283600938),
 ('JetEta_0', 0.029767274039328884),
 ('JetPhi_0', 0.029226050874977448),
 ('ZZPhi', 0.02733176979974743),
 ('JetPt_1', 0.024715857838715497),
 ('ExtraLepPt_1', 0.021288111131156414),
 ('ZZMassErr', 0.01614649106981779),
 ('ExtraLepPhi_0', 0.015605267905466354),
 ('JetPhi_1', 0.015154248601840158),
 ('ExtraLepEta_1', 0.013710986830236334),
 ('nExtraLep', 0.013350171387335378),
 ('JetEta_1', 0.012989355944434422),
 ('Z1Pt', 0.009832220819051055),
 ('ExtraLepEta_0', 0.00938120151542486),
 ('ExtraLepPhi_1', 0.003066931264658127),
 ('Z1Flav', 0.000270611

In [92]:
%%capture
fig, implist = plot_variables([("ZHMET", "ggH")])

In [93]:
implist

[('PFMET', 0.19808506209671825),
 ('ZZPt', 0.19551793519739125),
 ('JetPt_0', 0.15035037813085408),
 ('ZZPhi', 0.06813293554430029),
 ('Z2Pt', 0.05141191979462985),
 ('ZZMassErr', 0.04787344758204399),
 ('Z1Pt', 0.047040865884964964),
 ('ZZMass', 0.040449594116422675),
 ('Z1Mass', 0.03739679456046625),
 ('ZZEta', 0.03476028585304933),
 ('Z2Mass', 0.032817595226531604),
 ('JetPhi_0', 0.02310414209394297),
 ('JetEta_0', 0.022757233053493375),
 ('JetPt_1', 0.017622979254839383),
 ('nCleanedJetsPt30', 0.01422327065843336),
 ('JetPhi_1', 0.00707694442517172),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.005273017414833831),
 ('JetEta_1', 0.00409352667730521),
 ('Z1Flav', 0.0014570179698882952),
 ('Z2Flav', 0.0005550544647193506)]

In [94]:
%%capture
fig, implist = plot_variables([("ZHMET", "VBF")])

In [95]:
implist

[('PFMET', 0.24298758457597058),
 ('JetPt_0', 0.11101622544833475),
 ('JetEta_0', 0.10773172173684556),
 ('ZZPt', 0.10379031728305853),
 ('Z1Pt', 0.07002561912894961),
 ('Z2Mass', 0.05741312487683111),
 ('JetEta_1', 0.04723116337121461),
 ('ZZEta', 0.041910267358602114),
 ('Z2Pt', 0.03947973461210011),
 ('ZZPhi', 0.03606385075215135),
 ('ZZMassErr', 0.0321881363725941),
 ('ZZMass', 0.02896932273533469),
 ('Z1Mass', 0.026998620508441176),
 ('JetPhi_0', 0.024962228207317874),
 ('JetPhi_1', 0.013400775142875911),
 ('JetPt_1', 0.008408329501412336),
 ('nCleanedJetsPt30', 0.004204164750706168),
 ('Z1Flav', 0.0025619128949615714),
 ('Z2Flav', 0.00045983051960848714),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.00019707022268935164)]

In [96]:
%%capture
fig, implist = plot_variables([("ZHMET", "WHh")])

In [97]:
implist

[('PFMET', 0.2378608682876662),
 ('JetPt_0', 0.1053404833614927),
 ('ZZPt', 0.10005142143539264),
 ('JetEta_0', 0.07485491809299934),
 ('Z2Mass', 0.06559905972232424),
 ('Z1Pt', 0.06537868214207009),
 ('Z1Mass', 0.04135752589436568),
 ('Z2Pt', 0.04091677073385734),
 ('ZZPhi', 0.04062293396018512),
 ('ZZEta', 0.03908029089840594),
 ('ZZMass', 0.03665613751561008),
 ('ZZMassErr', 0.03599500477484757),
 ('JetPt_1', 0.035554249614339235),
 ('JetEta_1', 0.025710717696319694),
 ('nCleanedJetsPt30', 0.019393227062366854),
 ('JetPhi_0', 0.01792404319400573),
 ('JetPhi_1', 0.013075736428414015),
 ('Z1Flav', 0.0027914493498861384),
 ('Z2Flav', 0.0014691838683611254),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.00036729596709028135)]

In [98]:
%%capture
fig, implist = plot_variables([("ZHMET", "ttHh")])

In [99]:
implist

[('PFMET', 0.2668463611859838),
 ('nCleanedJetsPt30', 0.16112608565438755),
 ('JetPt_0', 0.11919736448038334),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.1111111111111111),
 ('JetEta_0', 0.07038035339922133),
 ('JetPt_1', 0.06199460916442048),
 ('ExtraLepPt_0', 0.03893381251871818),
 ('Z1Pt', 0.03533992213237496),
 ('ZZEta', 0.03294399520814615),
 ('Z2Mass', 0.023060796645702306),
 ('JetEta_1', 0.01737047020065888),
 ('ZZPt', 0.016771488469601678),
 ('Z2Pt', 0.011680143755615454),
 ('JetPhi_0', 0.011081162024558252),
 ('ZZMassErr', 0.009583707696915245),
 ('JetPhi_1', 0.005989817310572028),
 ('ZZMass', 0.005091344713986224),
 ('Z1Mass', 0.0011979634621144056),
 ('ZZPhi', 0.0002994908655286014)]

In [100]:
%%capture
fig, implist = plot_variables([("ZHMET", "ttHl")])

In [101]:
implist

[('ExtraLepPt_0', 0.2208686440677966),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.15148305084745764),
 ('nCleanedJetsPt30', 0.14989406779661016),
 ('JetPt_0', 0.1281779661016949),
 ('ZZMass', 0.06408898305084745),
 ('PFMET', 0.05429025423728814),
 ('nExtraLep', 0.045021186440677964),
 ('JetPt_1', 0.03389830508474576),
 ('Z1Mass', 0.023834745762711863),
 ('JetEta_0', 0.023305084745762712),
 ('ZZPhi', 0.023040254237288137),
 ('ZZEta', 0.022245762711864406),
 ('JetEta_1', 0.012976694915254237),
 ('ZZPt', 0.012711864406779662),
 ('ZZMassErr', 0.01059322033898305),
 ('Z1Pt', 0.010063559322033898),
 ('JetPhi_0', 0.004502118644067797),
 ('Z2Pt', 0.00423728813559322),
 ('Z2Mass', 0.00211864406779661),
 ('JetPhi_1', 0.00211864406779661),
 ('Z1Flav', 0.00026483050847457627),
 ('Z2Flav', 0.00026483050847457627)]

In [102]:
%%capture
fig, implist = plot_variables([("ttHh", "ggH")])

In [103]:
implist

[('JetPt_0', 0.11779271312345242),
 ('JetPt_1', 0.11743898125221082),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.11673151750972763),
 ('nCleanedJetsPt30', 0.11372479660417403),
 ('ZZPt', 0.08507251503360452),
 ('JetEta_1', 0.08330385567739654),
 ('JetEta_0', 0.08100459851432613),
 ('Z1Pt', 0.07339936328263176),
 ('PFMET', 0.04704633887513265),
 ('Z2Pt', 0.037141846480367884),
 ('ExtraLepPt_0', 0.029713477184294304),
 ('ZZMass', 0.020693314467633533),
 ('ZZEta', 0.019985850725150335),
 ('JetPhi_0', 0.016802263883975946),
 ('Z1Mass', 0.010435090201627167),
 ('ZZMassErr', 0.010081358330385568),
 ('ZZPhi', 0.007074637424831978),
 ('Z2Flav', 0.005305978068623983),
 ('JetPhi_1', 0.003714184648036788),
 ('Z2Mass', 0.002476123098691192),
 ('ExtraLepEta_0', 0.0010611956137247967)]

In [104]:
%%capture
fig, implist = plot_variables([("ttHh", "VBF")])

In [105]:
implist

[('JetEta_0', 0.17692272074825208),
 ('JetEta_1', 0.16293929712460065),
 ('nCleanedJetsPt30', 0.08394684446913923),
 ('JetPt_1', 0.08297448719729592),
 ('JetPt_0', 0.05482242904107052),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.0505625781358522),
 ('ZZEta', 0.04829374450155114),
 ('Z1Pt', 0.037921933601889155),
 ('PFMET', 0.03560679724035746),
 ('ZZPt', 0.035236375422512387),
 ('JetPhi_0', 0.03481965087743668),
 ('ZZMass', 0.03143955178960041),
 ('ExtraLepPt_0', 0.030235680881603926),
 ('Z2Mass', 0.027457517247765892),
 ('ZZPhi', 0.02491086725008103),
 ('ZZMassErr', 0.024725656341158496),
 ('Z1Mass', 0.022040098161781728),
 ('JetPhi_1', 0.01703940362087327),
 ('Z2Pt', 0.01097374635366023),
 ('ExtraLepPhi_0', 0.0033337963606056395),
 ('Z1Flav', 0.0014816872713802843),
 ('ExtraLepEta_0', 0.001157568180765847),
 ('Z2Flav', 0.001157568180765847)]

In [106]:
%%capture
fig, implist = plot_variables([("ttHh", "WHh")])

In [107]:
implist

[('nCleanedJetsPt30BTagged_bTagSF', 0.2367736339982654),
 ('nCleanedJetsPt30', 0.16305290546400694),
 ('JetPt_1', 0.10089621277825961),
 ('Z1Pt', 0.08701936976004626),
 ('JetPt_0', 0.07718993928881179),
 ('PFMET', 0.07198612315698179),
 ('ZZPt', 0.06938421509106678),
 ('ExtraLepPt_0', 0.04712344608268285),
 ('ZZEta', 0.037872217403873955),
 ('JetEta_1', 0.020526163631107257),
 ('Z2Pt', 0.015611448395490026),
 ('Z2Mass', 0.0132986412257878),
 ('ZZPhi', 0.012142237640936688),
 ('Z1Mass', 0.01185313674472391),
 ('JetEta_0', 0.010407632263660017),
 ('ZZMass', 0.009829430471234461),
 ('JetPhi_1', 0.005782017924255565),
 ('JetPhi_0', 0.004047412546978896),
 ('Z1Flav', 0.0026019080659150044),
 ('ZZMassErr', 0.0026019080659150044)]

In [108]:
%%capture
fig, implist = plot_variables([("ttHh", "ttHl")])

In [109]:
implist

[('PFMET', 0.16098127449557265),
 ('nCleanedJetsPt30', 0.08622441573523008),
 ('ExtraLepPt_0', 0.0833212367542459),
 ('JetPt_1', 0.06089417912614312),
 ('JetPhi_1', 0.054144287995354916),
 ('Z2Pt', 0.047249237915517495),
 ('ZZMass', 0.046450863695746844),
 ('ZZMassErr', 0.04637828422122224),
 ('Z1Mass', 0.04202351574974597),
 ('ZZPt', 0.04115256205545072),
 ('JetEta_0', 0.040862244157352304),
 ('JetPt_0', 0.040136449412106256),
 ('ZZEta', 0.03614457831325301),
 ('Z2Mass', 0.03200754826535056),
 ('Z1Pt', 0.030265640876760053),
 ('JetEta_1', 0.029104369284366383),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.026418928726956017),
 ('JetPhi_0', 0.024749600812890116),
 ('ZZPhi', 0.023951226593119466),
 ('ExtraLepEta_0', 0.023298011322398025),
 ('ExtraLepPhi_0', 0.02242705762810277),
 ('Z2Flav', 0.0007257947452460444),
 ('Z1Flav', 0.0005806357961968355),
 ('ExtraLepEta_1', 0.00021773842357381332),
 ('ExtraLepPhi_1', 0.00021773842357381332),
 ('nExtraLep', 7.257947452460444e-05)]

In [110]:
%%capture
fig, implist = plot_variables([("ttHl", "ggH")])

In [111]:
implist

[('PFMET', 0.1771714729753065),
 ('ZZPt', 0.07062683474356761),
 ('ExtraLepPt_0', 0.06807977896736316),
 ('JetPt_0', 0.0665688136763944),
 ('JetPt_1', 0.056035227076498016),
 ('nCleanedJetsPt30', 0.05391987566914177),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.05063892246589535),
 ('Z1Mass', 0.04722845795199447),
 ('JetEta_1', 0.04498359523398377),
 ('ZZMass', 0.04390433431186323),
 ('JetEta_0', 0.04373165256432395),
 ('ZZMassErr', 0.0426092212053186),
 ('Z1Pt', 0.039716801934035574),
 ('Z2Mass', 0.03198929373165257),
 ('ZZEta', 0.030262476256259715),
 ('ZZPhi', 0.030219305819374893),
 ('Z2Pt', 0.023873251597306164),
 ('JetPhi_0', 0.02227594543256778),
 ('JetPhi_1', 0.01895182179243654),
 ('nExtraLep', 0.013685028492488344),
 ('ExtraLepEta_0', 0.011396995337592817),
 ('Z1Flav', 0.007295803833534795),
 ('Z2Flav', 0.004230702814712485),
 ('ExtraLepPhi_0', 0.0006043861163874979)]

In [112]:
%%capture
fig, implist = plot_variables([("ttHl", "VBF")])

In [113]:
implist

[('PFMET', 0.12204944624678375),
 ('JetEta_0', 0.10940821120930753),
 ('JetEta_1', 0.09961964425550957),
 ('ExtraLepPt_0', 0.08149681172390648),
 ('nCleanedJetsPt30', 0.07422530484394227),
 ('JetPt_1', 0.06113659246000671),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.04877503076406757),
 ('JetPt_0', 0.04485960398254838),
 ('ZZMass', 0.042622217250251705),
 ('ZZPt', 0.04167132788902562),
 ('ZZEta', 0.03999328783980311),
 ('Z1Mass', 0.03328112764291308),
 ('ZZMassErr', 0.03199463027184249),
 ('ExtraLepEta_0', 0.031323414252153486),
 ('JetPhi_1', 0.030540328895849646),
 ('ZZPhi', 0.022206063318044523),
 ('Z2Mass', 0.020416153932207182),
 ('JetPhi_0', 0.018458440541447588),
 ('Z1Pt', 0.01605324980422866),
 ('Z2Pt', 0.014878621769772905),
 ('nExtraLep', 0.004418838796285938),
 ('Z2Flav', 0.004306969459671104),
 ('Z1Flav', 0.0034679494350598502),
 ('ExtraLepPhi_0', 0.0027967334153708467)]

In [114]:
%%capture
fig, implist = plot_variables([("ttHl", "WHh")])

In [115]:
implist

[('PFMET', 0.17378716192359298),
 ('ExtraLepPt_0', 0.09759197796765764),
 ('nCleanedJetsPt30', 0.08389238048160441),
 ('JetPt_0', 0.06969846762234305),
 ('nCleanedJetsPt30BTagged_bTagSF', 0.06419038203516701),
 ('Z1Mass', 0.04879598898382882),
 ('ZZEta', 0.04244050561401031),
 ('ZZMassErr', 0.0406750935668385),
 ('JetEta_0', 0.039898312266082904),
 ('Z2Pt', 0.038485982628345454),
 ('ZZPt', 0.03601440576230492),
 ('ZZMass', 0.035731939834757434),
 ('JetPt_1', 0.03509639149777558),
 ('JetPhi_1', 0.029305839983052043),
 ('JetEta_1', 0.02859967516418332),
 ('ZZPhi', 0.028105359790975212),
 ('Z1Pt', 0.024574535696631594),
 ('JetPhi_0', 0.022244191794364803),
 ('Z2Mass', 0.018360285290586822),
 ('ExtraLepEta_0', 0.01652425676152814),
 ('ExtraLepPhi_0', 0.010945554692465222),
 ('Z2Flav', 0.005366852623402302),
 ('nExtraLep', 0.004872537250194195),
 ('Z1Flav', 0.004801920768307323)]